# Saving the IABP data into one NetCDF file

In this Jupyter notebook, we convert the csv files containing the data of the IABP buoys into one NetCDF file in order to manipulate more easily the data in the following. For that, we first read the csv files and check that there is no obvious error that occurred when writing the files. Once it's done, we transform the time into seconds since 1970-01-01 00:00:00 and save our data into a NetCDF file.

**First step before everything else: downloading the IABP data!**

In our case only the year 1997 interests us. We download the data thanks to this command written in the terminal where we want the data to be:

wget -r -np --no-check-certificate --no-host-directories -R "index.html*" -R "*.dat" https://iabp.apl.uw.edu/Data_Products/BUOY_DATA/3HOURLY_DATA/1997/

**N.B.** We find an error in the 10667.csv file of the year 1997. The name of the buoy in the file (17987) is not the same as the file one (10667). We choose to merge the two names into one and create a new file: 1066717987.csv, which is identical to 10667.csv except that the id of the buoy is now 1066717987. We rename the 10667.csv file into 10667_pb.csv.

In [1]:
#Loading of the libraries useful in the following

import numpy as np
import xarray as xr
import pandas as pd
import cftime

##For reading the files
import glob

##For saving into an NetCDF file
from sitrack import ncio

In the Jupyter notebook, we use a function of sitrack (https://github.com/brodeau/sitrack) to save our NetCDF file. For the saving to work, the sitrack package should have been installed before running the notebook. Furthermore, this line must appear in the .bashrc, .profile or equivalent file of the used machine in order to enable Python to locate the modules: **export PYTHONPATH=<absolute_path_to_somewhere\>:${PYTHONPATH}** with <absolute_path_to_somewhere\> the path to the localisation of the sitrack package.

### Reading the CSV files

In [2]:
#Some useful variables to find the data
year_interest=1997
path="/Users/fioll/Documents/IABP_buoys/Data_Products_IABP/BUOY_DATA/3HOURLY_DATA/"+str(year_interest)+"/" #path to where the data is located
sep_charac=","

print(path,"column separation character: "+sep_charac)

/Users/fioll/Documents/IABP_buoys/Data_Products_IABP/BUOY_DATA/3HOURLY_DATA/1997/ column separation character: ,


In [3]:
#Loading the buoys files into one panda frame

all_files=glob.glob(path+"*.csv") #names of all the files in the folder corresponding of the year of interest
#print(len(all_files),all_files)

list_id_buoys_not_sorted=[] #list that will contain the id of the buoys (not sorted)
df=[] #list that will contain the different pandaframes before merging them

for filename in all_files: #iteration over all the files
    if "_pb.csv" in filename: #we disregard the files previously identified with a problem
        print(filename,"erroneous file, we do not consider it")
    else:
        ##storage of the id of the buoys
        list_id_buoys_not_sorted.append(int(str.split(str.split(filename,"/")[-1],".")[0]))

        ##loading of the csv files
        df_tmp=pd.read_csv(filename,sep=sep_charac)        

            ##Checking if the last two lines are the same: in that case we do not take into account the last one
        index_max=df_tmp.index.max()
        #print(df_tmp.loc[index_max-1]-df_tmp.loc[index_max])
        if np.sum(df_tmp.loc[index_max-1]-df_tmp.loc[index_max])==0.0:
            #print(filename)
            df.append(df_tmp[:-1])
        else:
            df.append(df_tmp)
        
##sorting the id of the buoys
list_id_buoys=np.sort(list_id_buoys_not_sorted)
#print(len(list_id_buoys_not_sorted),list_id_buoys_not_sorted)
print(len(list_id_buoys),list_id_buoys)

##merging all the pandaframes into one
merged_df=pd.concat(df,ignore_index=True)
    ##some quick checks
#print(merged_df.loc[merged_df.BuoyID==1103][:15])
#print(merged_df.loc[merged_df.BuoyID==26698][-15:])

/Users/fioll/Documents/IABP_buoys/Data_Products_IABP/BUOY_DATA/3HOURLY_DATA/1997/10667_pb.csv erroneous file, we do not consider it
40 [      1101       1103       1901       1902       1903       2385
       2417       5315       5317       7100       7105       8057
       8058       9353       9354       9357       9358       9360
       9361       9365      12795      12796      17987      19577
      19578      19579      20726      22204      22205      22206
      22207      24228      26693      26694      26695      26696
      26698      26699      26700 1066717987]


### Checking that there is no problem in the files

In [4]:
#Checking that the csv files seems okay

#for ib in range(len(list_id_buoys)): #iteration over the buoys
    #print(ib,list_id_buoys_not_sorted[ib])

    ##Checking concatenation okay for all the buoys
    #print((df[ib]-merged_df.loc[merged_df.BuoyID==list_id_buoys_not_sorted[ib]].set_index(df[ib].index)).sum())
    #=>okay
    
    ##Checking the id of the buoy within a file
    #print(df[ib].BuoyID.min(),df[ib].BuoyID.max())
    #=> problem with the file 10667.csv => fixed

    ##Checking the year within a file
    #print(df[ib].Year.min(),df[ib].Year.max()) #1997 (97) should be within the two values
    #print(df[ib].loc[df[ib].Year==96]) #should be in the last days of December c.a.d DOY >= 366 or about
    #print(df[ib].loc[df[ib].Year==98]) #should be in the first days of January c.a.d DOY < 2 or about
    #=> okay

    ##Checking if we correctly read the csv file: do not take into account the last line
    #index_max=df[ib].index.max()
    #print(index_max,df[ib].loc[[0,index_max]])
    #=>okay

    ##Checking if for a given buoy we do not have twice the same date
    #print(df[ib][["Year","DOY"]].duplicated().any())
    #=>okay
    
    ##Checking the hours: should be a multiple of 3 and 0<=hour<24
    #print(df[ib].loc[df[ib].Hour%3!=0])
    #print(df[ib].loc[df[ib].Hour<0])
    #print(df[ib].loc[df[ib].Hour>21])
    #=>okay

    ##Checking DOY: should be coherent with the hour and not below 1 and not above or equal to 366 (not a leap year) or 367 (leap year)
    #print(df[ib].loc[df[ib].DOY<1])
    #print(df[ib].loc[df[ib].DOY>367])
    #print(df[ib].loc[df[ib].DOY>366])
    #print(((df[ib].DOY-np.floor(df[ib].DOY))-df[ib].Hour/24).sum())
    #=>okay

    ##Checking the latitudes and longitudes
    #print(df[ib].Lat.min(),df[ib].Lat.max(),df[ib].loc[df[ib].Lat>90],df[ib].loc[df[ib].Lat<60])
    #print(df[ib].Lon.min(),df[ib].Lon.max(),df[ib].loc[df[ib].Lon>180],df[ib].loc[df[ib].Lon<-180])
    #=>okay

    #print("//")

### Transforming the dataframe into a xarray Dataset with the time and the id of the buoys as dimensions

In [5]:
#converting the year into real year (ex: 97 => 1997)
merged_df["Year"]=merged_df["Year"]+int(str(year_interest)[:2])*100
print(merged_df["Year"].min(),merged_df["Year"].max())

1996 1998


In [6]:
#for each line recreating the corresponding date from the variables: "Year", "DOY" and "Hour"
merged_df['Time'] = pd.Series(pd.to_datetime((np.floor(merged_df["Year"] * 1000 + merged_df["DOY"]) * 100 + merged_df["Hour"]).astype(int),
            format="%Y%j%H"))
merged_df

,BuoyID,Year,Hour,DOY,Lat,Lon,BP,Ts,Time
0,22207,1997,9,349.375,75.580,-150.514,-999.9,-999.900,1997-12-15 09:00:00
1,22207,1997,12,349.500,75.577,-150.519,-999.9,-999.900,1997-12-15 12:00:00
2,22207,1997,15,349.625,75.576,-150.525,-999.9,-999.900,1997-12-15 15:00:00
3,22207,1997,18,349.750,75.576,-150.531,-999.9,-999.900,1997-12-15 18:00:00
4,22207,1997,21,349.875,75.572,-150.522,-999.9,-999.900,1997-12-15 21:00:00
...,...,...,...,...,...,...,...,...,...
65830,26695,1997,12,243.500,81.126,0.000,1029.4,0.384,1997-08-31 12:00:00
65831,26695,1997,15,243.625,81.118,0.000,1029.4,0.373,1997-08-31 15:00:00
65832,26695,1997,18,243.750,81.110,0.000,1029.4,0.363,1997-08-31 18:00:00
65833,26695,1997,21,243.875,81.102,0.000,1029.4,0.352,1997-08-31 21:00:00


In [7]:
#creating a new variable equal to 1
#this variable will tell us after the conversion into a xarray Dataset if we have data for a given buoy at a given date
merged_df["mask"] = np.repeat(1,merged_df.index.max()+1)
print(merged_df["mask"].isna().sum(),merged_df["mask"].sum())
merged_df

0 65835


,BuoyID,Year,Hour,DOY,Lat,Lon,BP,Ts,Time,mask
0,22207,1997,9,349.375,75.580,-150.514,-999.9,-999.900,1997-12-15 09:00:00,1
1,22207,1997,12,349.500,75.577,-150.519,-999.9,-999.900,1997-12-15 12:00:00,1
2,22207,1997,15,349.625,75.576,-150.525,-999.9,-999.900,1997-12-15 15:00:00,1
3,22207,1997,18,349.750,75.576,-150.531,-999.9,-999.900,1997-12-15 18:00:00,1
4,22207,1997,21,349.875,75.572,-150.522,-999.9,-999.900,1997-12-15 21:00:00,1
...,...,...,...,...,...,...,...,...,...,...
65830,26695,1997,12,243.500,81.126,0.000,1029.4,0.384,1997-08-31 12:00:00,1
65831,26695,1997,15,243.625,81.118,0.000,1029.4,0.373,1997-08-31 15:00:00,1
65832,26695,1997,18,243.750,81.110,0.000,1029.4,0.363,1997-08-31 18:00:00,1
65833,26695,1997,21,243.875,81.102,0.000,1029.4,0.352,1997-08-31 21:00:00,1


In [8]:
#Changing the index in order to have buoy id and the time as indexes

##Creating for each line the tuple corresponding to the new indexes
index_tuples=[]
for idx in range(merged_df.index.max()+1):
    index_tuples.append((merged_df.Time[idx],merged_df.BuoyID[idx]))
#print(index_tuples)

##Creating the new indexes
new_index = pd.MultiIndex.from_tuples(index_tuples, names=["time", "buoy"])
#print(new_index)

##Changing the indexes of merged_df
merged_df=merged_df.set_index(new_index)
merged_df

,,BuoyID,Year,Hour,DOY,Lat,Lon,BP,Ts,Time,mask
time,buoy,,,,,,,,,,
1997-12-15 09:00:00,22207,22207,1997,9,349.375,75.580,-150.514,-999.9,-999.900,1997-12-15 09:00:00,1
1997-12-15 12:00:00,22207,22207,1997,12,349.500,75.577,-150.519,-999.9,-999.900,1997-12-15 12:00:00,1
1997-12-15 15:00:00,22207,22207,1997,15,349.625,75.576,-150.525,-999.9,-999.900,1997-12-15 15:00:00,1
1997-12-15 18:00:00,22207,22207,1997,18,349.750,75.576,-150.531,-999.9,-999.900,1997-12-15 18:00:00,1
1997-12-15 21:00:00,22207,22207,1997,21,349.875,75.572,-150.522,-999.9,-999.900,1997-12-15 21:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...
1997-08-31 12:00:00,26695,26695,1997,12,243.500,81.126,0.000,1029.4,0.384,1997-08-31 12:00:00,1
1997-08-31 15:00:00,26695,26695,1997,15,243.625,81.118,0.000,1029.4,0.373,1997-08-31 15:00:00,1
1997-08-31 18:00:00,26695,26695,1997,18,243.750,81.110,0.000,1029.4,0.363,1997-08-31 18:00:00,1


In [9]:
#Dropping the columns that we don't need anymore
light_merged_df=merged_df.drop(labels=["BuoyID","Year","Hour","DOY","BP","Ts","Time"],axis=1,inplace=False)
print(np.shape(merged_df),merged_df)
print(np.shape(light_merged_df),light_merged_df)

(65835, 10)                            BuoyID  Year  Hour      DOY     Lat      Lon  \
time                buoy                                                  
1997-12-15 09:00:00 22207   22207  1997     9  349.375  75.580 -150.514   
1997-12-15 12:00:00 22207   22207  1997    12  349.500  75.577 -150.519   
1997-12-15 15:00:00 22207   22207  1997    15  349.625  75.576 -150.525   
1997-12-15 18:00:00 22207   22207  1997    18  349.750  75.576 -150.531   
1997-12-15 21:00:00 22207   22207  1997    21  349.875  75.572 -150.522   
...                           ...   ...   ...      ...     ...      ...   
1997-08-31 12:00:00 26695   26695  1997    12  243.500  81.126    0.000   
1997-08-31 15:00:00 26695   26695  1997    15  243.625  81.118    0.000   
1997-08-31 18:00:00 26695   26695  1997    18  243.750  81.110    0.000   
1997-08-31 21:00:00 26695   26695  1997    21  243.875  81.102    0.000   
1997-09-01 00:00:00 26695   26695  1997     0  244.000  81.093    0.000   

            

In [10]:
#Converting our panda DataFrame into an xarray Dataset
ds=light_merged_df.to_xarray()
ds

<xarray.Dataset>
Dimensions:  (time: 2935, buoy: 40)
Coordinates:
  * time     (time) datetime64[ns] 1996-12-30T21:00:00 ... 1998-01-01T15:00:00
  * buoy     (buoy) int64 1101 1103 1901 1902 ... 26698 26699 26700 1066717987
Data variables:
    Lat      (time, buoy) float64 75.04 nan nan nan nan ... nan nan nan nan nan
    Lon      (time, buoy) float64 -150.6 nan nan nan nan ... nan nan nan nan nan
    mask     (time, buoy) float64 1.0 nan nan nan nan ... nan nan nan nan nan

In [11]:
#The mask is equal to 0 for the dates when we have no data for a given buoy
print(np.isnan(ds["mask"]).sum())
ds["mask"]=ds["mask"].fillna(0) #convert the NaN into zeros
print(np.isnan(ds["mask"]).sum())
ds

<xarray.DataArray 'mask' ()>
array(51565)
<xarray.DataArray 'mask' ()>
array(0)


<xarray.Dataset>
Dimensions:  (time: 2935, buoy: 40)
Coordinates:
  * time     (time) datetime64[ns] 1996-12-30T21:00:00 ... 1998-01-01T15:00:00
  * buoy     (buoy) int64 1101 1103 1901 1902 ... 26698 26699 26700 1066717987
Data variables:
    Lat      (time, buoy) float64 75.04 nan nan nan nan ... nan nan nan nan nan
    Lon      (time, buoy) float64 -150.6 nan nan nan nan ... nan nan nan nan nan
    mask     (time, buoy) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [15]:
#Small trick to be able after to have our dates in seconds since 1997-01-01 00:00:00
#/!\ For this cell and the following ones to work, the folder "Some_files_produced" should exist, or change the name of the written file
ds.to_netcdf("Some_files_produced/file_tmp.nc")
ds.close()
ds = xr.open_dataset("Some_files_produced/file_tmp.nc", use_cftime=True).load()
ds.close()
ds

<xarray.Dataset>
Dimensions:  (time: 2935, buoy: 40)
Coordinates:
  * time     (time) object 1996-12-30 21:00:00 ... 1998-01-01 15:00:00
  * buoy     (buoy) int64 1101 1103 1901 1902 ... 26698 26699 26700 1066717987
Data variables:
    Lat      (time, buoy) float64 75.04 nan nan nan nan ... nan nan nan nan nan
    Lon      (time, buoy) float64 -150.6 nan nan nan nan ... nan nan nan nan nan
    mask     (time, buoy) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [16]:
#Converting our time into seconds since 1970-01-01 00:00:00
ds["time"]=cftime.date2num(ds.time,"seconds since 1970-01-01 00:00:00",has_year_zero=False,calendar="gregorian",)
ds

<xarray.Dataset>
Dimensions:  (time: 2935, buoy: 40)
Coordinates:
  * time     (time) int64 851979600 851990400 852001200 ... 883656000 883666800
  * buoy     (buoy) int64 1101 1103 1901 1902 ... 26698 26699 26700 1066717987
Data variables:
    Lat      (time, buoy) float64 75.04 nan nan nan nan ... nan nan nan nan nan
    Lon      (time, buoy) float64 -150.6 nan nan nan nan ... nan nan nan nan nan
    mask     (time, buoy) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

### Saving the buoys in a NetCDF file

In [17]:
ncio.ncSaveCloudBuoys("Some_files_produced/IABP_buoys_"+str(year_interest)+".nc",ds.time, ds.buoy,ds.Lat,ds.Lon,mask=ds.mask)


 *** [ncSaveCloudBuoys]: About to generate file: Some_files_produced/IABP_buoys_1997_test.nc ...
      ===> Some_files_produced/IABP_buoys_1997_test.nc saved!


0